# Exploring how Lichess' players spend their playing time

*Chess addiction has always been a recurring subject online, and while its formal existence should be left to rigourous studies, it has lead me to wonder more generally about how players spent their time playing on Lichess, which is the topic I will delve into today.*

The data was aggregated from the January 2023 Lichess rated games (freely available on the [Lichess database website](https://database.lichess.org)). Bots and games shorter than 4 plies were filtered out. For each account (which from now on I will consider equivalent to one user), I kept their their username, as well as for each time control (TC):

*   Their number of games
*   Their average rating
*   The "Approximate time spent", which is the total time they spent playing this TC, with each game duration computed from the starting clock and increment, [using Lichess' formula](https://lichess.org/faq#time-controls) to differenciate between blitz/bullet/etc: (**clock initial time in seconds) + 40 × (clock increment)**.
*   The "Real time spent",  which is the total time they spent playing this TC, based on the time left on the clock at the end of the game, and taking into account increment of course. It does not take into account the use of the `+15s` button, which was considered negligible.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Define global style of the plt for all graphs
# /!\ Note that using BOTH style is important (in this order)
plt.style.use("seaborn-v0_8-dark")
plt.style.use("dark_background")
plt.rcParams['axes.facecolor'] = "#444445" # "#444445" # darker is "#262421"
plt.rcParams["figure.facecolor"] = "#333" #"#333" # "#darker is "#161612"
plt.rcParams["grid.color"] = "#fff"
plt.rcParams["grid.linestyle"] = "-."
plt.rcParams["grid.linewidth"] = 0.2
plt.rcParams["axes.grid"] = True

DFT_HEIGHT = 4.8 # inches
DFT_WIDTH = 6.4 # inches

perfs = ["ultrabullet", "bullet", "blitz", "rapid", "classical"]
COLOR_MAP = { "ultrabullet": "#fdb462", "bullet": "#81b1d2", "blitz": "#fa8174", "rapid":"#b3de69", "classical":"#bfbbd9"}

In [ ]:
dtypes = {"username": "string"}

#for perf in perfs:
#    dtypes[f"{perf}_approximate_time"] = "Int64"
#    dtypes[f"{perf}_real_time"] = "Int64"
#    dtypes[f"{perf}_avg_rating"] = "Int64"

def restrict_to_perf(perf: str, with_error = True):
    l = ["username", f"{perf}_games", f"{perf}_avg_rating",f"{perf}_approximate_time", f"{perf}_real_time"]
    if with_error:
        l.append(f"{perf}_relative_error")
    return l

#df = pd.read_csv("time-spent.csv",dtype=dtypes)
df = pd.read_csv("time-spent-2023-01.csv",dtype=dtypes)
#df.dtypes

In [ ]:
df_time = df.copy()

def convert(c):
    df_time[c] = df_time[c].apply(lambda x: pd.to_timedelta(x,unit="S"))

for perf in perfs:
    convert(f"{perf}_approximate_time")
    convert(f"{perf}_real_time")
#df_time

Here is an extract from the data, restricted to bullet due to layout constraints.

In [ ]:
table_df = df_time[df_time["bullet_games"] > 2].iloc[:5][restrict_to_perf("bullet",False)]
#table_df.rename(columns={c:  c.replace("bullet_","") for c in restrict_to_perf("bullet",False)},
#          inplace=True, errors='raise')

#print(table_df.to_markdown())

|    | username            |   games |   avg_rating | approximate_time   | real_time       |
|---:|:--------------------|--------:|-------------:|:-------------------|:----------------|
|  0 | ange_de_la_mode     |      85 |         1430 | 0 days 01:32:20    | 0 days 02:26:59 |
|  5 | Ankit_khandelwal_07 |     214 |         1316 | 0 days 03:39:00    | 0 days 05:33:30 |
|  8 | jpn26               |     550 |         1577 | 0 days 09:10:00    | 0 days 14:07:11 |
| 10 | ptitbill            |     303 |         1465 | 0 days 13:28:00    | 0 days 20:37:07 |
| 17 | Inkety              |      31 |         1467 | 0 days 00:31:00    | 0 days 00:42:31 |

They were ~~2,073,933~~ 2,073,569 (removing bots) distinct users who played that month!

In [ ]:
#df_time.describe()

## How is time spend playing dispatched by time control?

In [ ]:
def sum_per_perf_of_col(df, column: str):
    res = []
    for p in perfs:
        res.append(df[column.format(p)].sum())
    return res

def count_per_perf_of_col(df, column: str):
    res = []
    for p in perfs:
        res.append(df[column.format(p)].count())
    return res

players = count_per_perf_of_col(df, "{}_games")
games = sum_per_perf_of_col(df,"{}_games")
real_time = sum_per_perf_of_col(df,"{}_real_time")
approx_time = sum_per_perf_of_col(df,"{}_approximate_time")
datas = [{
    "title": "Players",
    "data": players
},{
    "title": "Number of games",
    "data": games
},{
    "title": "Real time played",
    "data": real_time
},{
    "title": "Approximate time played",
    "data": approx_time
}]

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(DFT_WIDTH * 2, DFT_HEIGHT* 2))
axs = axs.flatten()
for ax, data in zip(axs, datas):
    ax.pie(data["data"],labels=perfs,colors=COLOR_MAP.values(),autopct="%d%%")
    ax.set_title(f"{data['title']} per time control")

As expected the number of games alone is not a good appreciation of how much total time is spent playing. On the other hand though, we can see that the approximate time formula is quite close to the truth for all but classical TC.

## In which time control do individual players spend the more time?

In [ ]:
def real_time_box_plot(ax, df, perfs):
    data = df[[f"{x}_real_time" for x in perfs]]
    data_dropna = [values.dropna() for (_, values) in data.items()]
    colors = [COLOR_MAP[x] for x in perfs]
    bplot = plt.boxplot(data_dropna,patch_artist=True,sym='',labels=perfs,showmeans=True,meanprops={ "markerfacecolor":"white", "markeredgecolor":"white"})
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)
fig = plt.figure(figsize=(DFT_WIDTH * 8 / 5, DFT_WIDTH))
ax = fig.add_subplot(1,1,1,)#nrows=1,)
datafile = 'user-blog-default.png'
img = plt.imread(datafile)
real_time_box_plot(ax,df,perfs)
print(ax.get_xlim())
xl, xh = ax.get_xlim()
yl, yh = ax.get_ylim()
ax.imshow(img, zorder=0,alpha=0.5,origin="upper",extent=[xl,xh, yl, yh],aspect="auto")
ax.set_ylabel("real time spent playing (s)")
fig.suptitle("Real time spent playing distribution")
fig.set_tight_layout(True) # for the blog post head image

This graph demonstrates that the significant popularity of blitz in the dataset is not only due to the fact that there are more players compared to say bullet or rapid, but also because, on average (shown by the triangle in the figure) and median, blitz players spend more time playing it than other time controls. Another point to consider regarding the average time spent is that it frequently exceeds the 75th percentile, indicating the presence of significant outliers (extremely active players in our case).

In [ ]:
df1 = df[restrict_to_perf("bullet",False)].copy()
df1.sort_values("bullet_real_time", ascending=False, inplace=True)
#df1.iloc[:20]

Now that we have a first overview of the time spent by time control, it's interesting to consider how in a specific TC the data is distributed.

In [ ]:
fig, axs = plt.subplots(ncols=2,nrows=2,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2 / 1.5))
axs = axs.flatten()
def hist(df, ax, data, color,perf,log: bool):
    ax.hist(df[data["column"]].astype("float64"),bins=100,color=color,log=log,label=perf,alpha=1)
    ax.set_xlabel(data["xlabel"])
    ax.set_ylabel("number of players")

def perf_hist(df, axs, perf: str,include_avg=False,log=False):
    datas = [{
        "column": f"{perf}_games",
        "xlabel": "number of games",
    }]
    if include_avg:
        datas.append({
       "column": f"{perf}_avg_rating",
       "xlabel": "average rating",
        })
    datas.extend([
    {
        "column": f"{perf}_approximate_time",
        "xlabel": "approximate time (s)",
    },{
        "column": f"{perf}_real_time",
        "xlabel": "real time (s)",
    }])
    for (ax, data) in zip(axs, datas):
        hist(df,ax,data,COLOR_MAP[perf],perf,log=log)

perf_hist(df1, axs, "bullet",include_avg=True)
fig.suptitle(f"Bullet distribution by")

The distribution of the number of games and time appears to follow a tail distribution, while the rating distribution conforms to the expected normal distribution. A big surprise compared to the [online rating distribution, which only take into account players with a stable rating](https://lichess.org/stat/rating/distribution/bullet) is the prominence of new players (big spike around 1500).

Let's see whether the tail distribution is also observed in the other time controls, using with a logarithmic y-axis this time.

In [ ]:
fig, axs = plt.subplots(ncols=3,nrows=len(perfs),figsize=(DFT_WIDTH * 2.5, DFT_HEIGHT * len(perfs) / 1.5))

for perf, axs_row in zip(perfs, axs):
    perf_hist(df, axs_row,perf,log=True)
    axs_row[0].legend()
fig.suptitle(f"Distribution by time control of",y=0.92,size="x-large")

While this overview clearly demonstrate all follow the same law, the lack of common scale make it difficult to compare them.

In [ ]:
def smooth(df, col,moving_average):
    df[col] = df[col].rolling(moving_average).mean()

# https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-for-a-dataset
def smoothSO(df, col, box_pts):
    box = np.ones(box_pts)/box_pts
    df[col] = np.convolve(df[col], box, mode='same')

def lowess(df, col_y, col_x, frac):
    #x = np.random.uniform(low = -2*np.pi, high = 2*np.pi, size=500)
    y = df[col_y]
    x = df[col_x]
    lowess = sm.nonparametric.lowess(endog=y, exog=x,frac=frac)
    df[col_x] = lowess[:, 0]
    df[col_y] = lowess[:, 1]
    

def ln(df, col):
    df[col] = np.log(df[col])

def exp(df, col):
    df[col] = np.exp(df[col])

def get_distrib(*, df, col,bins, frac: int | None):
    distribs = [np.histogram(df[col.format(perf)].dropna(),bins=bins) for perf in perfs]
    distrib_plot = []
    for distrib in distribs:
        # we want to compute the average of the bucket
        average_x = pd.DataFrame(distrib[1], columns=["bucket"])["bucket"].rolling(2).mean().dropna()
        res = pd.DataFrame(zip(average_x, distrib[0]), columns=['avg_buckets', 'nb_players'])
        # drop points with 0 players
        res = res[res["nb_players"] != 0]
        if frac:
            lowess(res, col_y="nb_players", col_x="avg_buckets",frac=frac)
        distrib_plot.append(res)
    return distrib_plot

def draw_loglog(*, ax, df, col,bins,frac):
    for perf, plot_df in zip(perfs, get_distrib(df=df, col=col,bins=bins,frac=frac)):
        ax.loglog(plot_df["avg_buckets"], plot_df["nb_players"],color=COLOR_MAP[perf],label=perf)
    #ax.set_yscale('log')
    #ax.set_xscale('log')
    ax.set_ylabel("number of players")
    ax.legend()

fig, ax = plt.subplots(ncols=2,nrows=1, figsize=(DFT_WIDTH * 2, DFT_HEIGHT))

draw_loglog(ax=ax[0], df=df, col="{}_games",bins=100,frac=0.1)
ax[0].set_title("number of games distribution (LOWESS Smoothing)")
ax[0].set_xlabel("number of games played")

draw_loglog(ax=ax[1], df=df, col="{}_real_time",bins=10_000,frac=0.011)
ax[1].set_title("real time spent playing distribution (LOWESS Smoothing)")
ax[1].set_xlabel("real time spent playing (s)")

This loglog graph confirms the exponential decrease of players per number of games played. 

Additionally, there is a noticeable spike in game duration for each time control, aligning with the expected duration of one game in that specific TC. Interestingly, as the TC becomes slower, the spike in game duration becomes smoother, indicating a greater variety in game duration.

However, what is peculiar is that this pattern does not hold true for classical time control. Unlike bullet, blitz, and rapid, where longer time controls result in longer game durations, players seem to blitz-out more than they should in classical games, thus finishing them quicker than expected.

In [ ]:
# Compute the relative error of the real time spent, compared to the average computed one
df_avg_error = df.copy()
for perf in perfs:
    df_avg_error[f"{perf}_relative_error"] = (df[f"{perf}_approximate_time"] - df[f"{perf}_real_time"]).abs() / df[f"{perf}_real_time"]

# Restricting to blitz to see all columns
#df_avg_error[restrict_to_perf("blitz")]

## How accurate is the approximate game duration formula?

Let's now look a bit deeper on the approximate game duration formula used by Lichess and see if it confirms our hunch that at least for classical it is too generous.

In [ ]:
avg_errors = [f"{p}_relative_error" for p in perfs]
def filter_relative_error(df, perfs):
    return [df[df[f"{c}_relative_error"] < df[f"{c}_relative_error"].quantile(0.99)][f"{c}_relative_error"] for c in perfs]

def relative_better_boxplot(ax, df, perfs):
    data = filter_relative_error(df, perfs)
    colors = [COLOR_MAP[x] for x in perfs]
    bplot = ax.boxplot(data,patch_artist=True,sym='',labels=perfs,showmeans=True,meanprops={ "markerfacecolor":"white", "markeredgecolor":"white"})
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)

    
fig, axs = plt.subplots(ncols=2,nrows=1,figsize=(DFT_WIDTH * 1.5, DFT_HEIGHT),gridspec_kw={"width_ratios": [4, 1]})
relative_better_boxplot(axs[0],df_avg_error,perfs[:-1])
y_label = "absolute relative error"
axs[0].set_ylabel(y_label)
relative_better_boxplot(axs[1],df_avg_error,perfs[-1:])
axs[1].set_ylabel(y_label)
fig.suptitle("Game duration approximation absolute relative error")
#relative_better_boxplot(axs[0], df_avg_error, perfs)

As we can see the formula is quite accurate for short time controls, but its accuracy diminishes as the initial time on the clock increases, to the point that a separate subplot was required for classical in order to accommodate the change in scale.

But does the formula overestimate or underestimate the game duration? To answer that a question we need to look at the non-absolute relative error distribution.

In [ ]:
non_abs_rel_error = df.copy()

APPROX_ERROR_MIN_QUANTILE = 0.01
APPROX_ERROR_MAX_QUANTILE = 0.95
for perf in perfs:
    non_abs_rel_error[f"{perf}_relative_error"] = (df[f"{perf}_approximate_time"] - df[f"{perf}_real_time"]) / df[f"{perf}_real_time"]


fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2))
axs = axs.flatten()

def perf_hist_err(df, ax, perf: str):
    col = f"{perf}_relative_error"
    data = df[ (df[col] < df[col].quantile(APPROX_ERROR_MAX_QUANTILE)) & (df[col] > df[col].quantile(APPROX_ERROR_MIN_QUANTILE))][col]
    ax.hist(data,bins=100,color=COLOR_MAP[perf],label=perf)
    ax.set_xlabel("relative error")
    ax.set_ylabel("number of players")
    ax.legend()

for perf, ax in zip(perfs, axs):
    perf_hist_err(non_abs_rel_error, ax,perf)
axs[-1].axis('off')
fig.suptitle(f"Game duration formula relative error (quantiles {APPROX_ERROR_MIN_QUANTILE}-{APPROX_ERROR_MAX_QUANTILE})",y=0.92,size="x-large")

The formula tends to underestimate the game duration in short time controls, then starts overestimating it. Note that outliers were removed from the data visualization. I am not sure what phenomon create these spikes int the ultrabullet distribution, but I assume it has to do with the fact I only have game duration down to the second, which may not accurately capture the time taken in those ultra-fast games.


In [ ]:
#df_avg_error[[f"{p}_relative_error" for p in perfs]].describe(percentiles=[.5, .75,.90,.95,.99])

## Do better players play more?

While correlation is not causality, it is interesting to see if better players play more than the average. A first approach is to graph the time spent playing per rating and see if a trend can be seen at the naked eye.

In [ ]:
# Scatter plot, x rating, y time-spent

def scatter(df, ax, perf):
    ax.scatter(x=df[f"{perf}_avg_rating"],y=df[f"{perf}_real_time"],color=COLOR_MAP[perf],s=1)
    ax.set_ylabel("real time (s)")
    #ax.set_label(perf)
    ax.set_xlabel("rating")

fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2))
axs = axs.flatten()
for ax, perf in zip(axs, perfs):
    scatter(df, ax, perf)
    ax.legend(labels=[perf],markerscale=5,markerfirst=False) # FIXME: couldn't get to make the legend box appear 
# last one is all combined
for perf in perfs:
    scatter(df, axs[-1], perf)
axs[-1].legend(labels=perfs,markerscale=5,markerfirst=False) # FIXME: couldn't get to make the legend box appear 
fig.suptitle("Real time played per rating",y=0.92,size="x-large")

There seems to be a slight tendency for players above 1500 to play more, we can confirm that by checking the rating distribution weighted by time played.

In [ ]:
# Weighted histogram. Total time per rating

fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2))
axs = axs.flatten()

def perf_hist_weighted(df, ax, perf: str):
    ax.hist(df[f"{perf}_avg_rating"],bins=100,color=COLOR_MAP[perf],weights=df[f"{perf}_real_time"],density=True, label=f"weighted {perf}")
    ax.hist(df[f"{perf}_avg_rating"],bins=100,alpha=0.4,density=True,edgecolor = "black",color="white",label=perf) # color=COLOR_MAP[perf]
    ax.set_xlabel("rating")
    ax.set_ylabel("density")
    ax.legend()
    #ax.set_title(perf)

for perf, ax in zip(perfs, axs):
    perf_hist_weighted(df, ax,perf)
axs[-1].axis('off')
fig.suptitle("rating density, weighted by time played",y=0.92,size="x-large")

Proportionally weighting by time played has the consequence of erasing the new players spike at 1500 in all but classical time controls, as well as shifting the bell curve to the right (compared to the online version), confirming that **better players (as in players above the median) play more games**, an expected result.

A last question that can be asked is if those highly active players participate in multiple time controls or if they specialize in just one.

In [ ]:
from PIL import Image, ImageOps


# https://stackoverflow.com/a/60737369/11955835
def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, int(delta_width - pad_width), delta_height - pad_height)
    return ImageOps.expand(img, padding)

# Make it so all images have 1.33 aspect ratio
def normalise_img(path):
    desired_aspect_ratio = 1.33
    old = Image.open(path)
    width, height = old.size
    current_asp = width / height
    if abs(current_asp - desired_aspect_ratio) < 0.01:
        return old # Good enough
    return resize_with_padding(old, (int(width * desired_aspect_ratio), int(width)))

def perf_to_perf_corr_matrix(df, fig, ax, perf1, perf2):
    ax.set_xticks([])
    ax.set_yticks([])
    if perf1 == perf2:
        img = np.array(normalise_img(f"perfs-icon/{perf1}.png"))
        ax.imshow(img)
        ax.set_facecolor(fig.get_facecolor())
        return
    norm = mpl.colors.Normalize(vmin=-1, vmax=1)
    # between -1 and 1
    # Confirmed that it only take into account rows where's no `Nan`
    corr = df[f"{perf1}_real_time"].corr(df[f"{perf2}_real_time"])
    ax.text(0.5, 0.5, f"{corr:.2f}", horizontalalignment='center', verticalalignment='center',fontsize=14, weight='bold',color="black")
    color = plt.cm.seismic(norm(corr))
    # Create a color gradient based on value
    # Set the background color of the figure
    ax.patch.set_facecolor(color)

def cartesian_perf():
    for p1 in perfs:
        for p2 in perfs:
            yield (p1, p2)
#mother_fig = plt.figure(layout='constrained', figsize=(DFT_WIDTH * 1.5, DFT_HEIGHT * 1.5))
#subfigs = mother_fig.subfigures(ncols=2, nrows=1, width_ratios=[10,1])
#fig = subfigs[0]
#axs = fig.subplots(ncols=len(perfs),nrows=len(perfs))
#axs = axs.flatten()
#fig.subplots_adjust(wspace=0, hspace=0)

def draw_perf_to_perf_corr_matrix(df, axs, fig):
    for i, (ax, cart_perf) in enumerate(zip(axs, cartesian_perf())):
        perf_to_perf_corr_matrix(df, fig, ax, *cart_perf)
        if i % 5 == 0: # first column
            ax.set_ylabel(cart_perf[0])
        else:
            ax.set_yticklabels([])
        if i >= 20: # last row
            ax.set_xlabel(cart_perf[1])
        else:
            ax.set_xticklabels([])
    
    fig.suptitle("Scatter matrix plot of time spent by time control",y=0.92,size="x-large")
    label = "real time spent (s)"
    fig.supxlabel(label)#size="x-large")#y=0.07)
    fig.supylabel(label)#size="x-large")#x=0.09)
#draw_perf_to_perf_corr_matrix(df, axs, fig)
plt.matshow(df[[f"{perf}_real_time" for perf in perfs]].corr())
#df[[f"{perf}_real_time" for perf in perfs]].corr()

Clearly active players seem to be focused on one particular time control most of the time, and the further apart the different time controls are in terms of speed, the fewer players we find playing both of them, with the extreme example being ultrabullet & classical players.

## Conclusion

If you feel trapped in an unstoppable blitz frenzzy, you are not alone, but I guess you did not need those numbers to know that ;)

In [ ]:
df_total = df.copy()
df_total["total_real_time"] = df[[f"{perf}_real_time" for perf in perfs]].sum(axis=1)
df_total["total_formated"] = df_total["total_real_time"].apply(lambda x: pd.to_timedelta(x,unit="S"))
df_total.sort_values("total_real_time", ascending=False, inplace=True)
df_total[["username", "total_formated"]].iloc[:5]

In [ ]:
def plot_cum_perfs(ax, df):
    usernames = df["username"]
    time_played_acc = np.zeros(len(usernames))
    for perf in perfs:
        data = df[f"{perf}_real_time"].fillna(0)
        rects = ax.barh(usernames, data, color=COLOR_MAP[perf],left=time_played_acc,label=perf)
        time_played_acc += data
    assert time_played_acc.equals(df["total_real_time"])

    ax.bar_label(rects, df["total_formated"],
                  padding=-100, color='white', fontweight='bold')
    ax.legend()
    ax.invert_yaxis() # to have the top value at the top
    ax.set_xlim(right=time_played_acc.max() * 1.3)
    
fig = plt.figure(figsize=(DFT_WIDTH * 1.5, DFT_HEIGHT * 1.5))
ax = fig.add_subplot(1,1,1,)
top = 20
plot_cum_perfs(ax, df_total.iloc[:top])
ax.set_title(f"Top {top} most active players")
#ax.set_ylabel("usernames")
ax.set_xlabel("real time (s)")
#ax.barh(["a", "b", "c"], [1, 2, 3])
#plt.show()